# How-to Finetune

**Before we start**

- This tutorial is rendered from a Jupyter notebook that is hosted on GitHub. If you want to run the code yourself, you can find the notebook and configuration files [here](https://github.com/neuralhydrology/neuralhydrology/tree/master/examples/06-Finetuning).
- To be able to run this notebook locally, you need to download the publicly available CAMELS US rainfall-runoff dataset and a publicly available extensions for hourly forcing and streamflow data. See the [Data Prerequisites Tutorial](data-prerequisites.nblink) for a detailed description on where to download the data and how to structure your local dataset folder. Note the special [section](data-prerequisites.nblink#CAMELS-US-catchment-attributes) with additional requirements for this tutorial.

This tutorial shows how to adapt a pretrained model to a different, eventually much smaller dataset, a concept called finetuning. Finetuning is well-established in machine learning and thus nothing new. Generally speaking, the idea is to use a (very) large and diverse dataset to learn a general understanding of the underlying problem first and then, in a second step, adapt this general model to the target data. Usually, especially if the available target data is limited, pretraining plus finetuning yields (much) better results than only considering the final target data. 

The connection to hydrology is the following: Often, researchers or operators are only interested in a single basin. However, considering that a Deep Learning (DL) model has to learn all (physical) process understanding from the available training data, it might be understandable that the data records of a single basin might not be enough (see e.g. the presentation linked at [this](https://meetingorganizer.copernicus.org/EGU2020/EGU2020-8855.html) EGU'20 abstract)

This is were we apply the concept of pretraining and finetuning: First, we train a DL model (e.g. an LSTM) with a large and diverse, multi-basin dataset (e.g. CAMELS) and then finetune this model to our basin of interest. Everything you need is available in the NeuralHydrology package and in this notebook we will give you an overview of how to actually do it.

**Note**: Finetuning can be a tedious task and is usually very sensitive to the learning rate as well as the number of epochs used for finetuning. One reason is that the pretrained models are usually quite large. In fact, most often they are much larger than what would be possible to train for just a single basin. So during finetuning, we have to make sure that this large capacity is not negatively impacting our model results. Common approaches are to a) only allow parts of the model to be adapted during finetuning and/or b) to train with a much lower learning rate. So far, no publication was published that presents a universally working approach for finetuning in hydrology. So be aware that the results may vary and you might need to invest some time before finding a good strategy. However, in our experience it was always possible to get better results _with_ finetuning than without.

**To summarize**: If you are interested in getting the best-performing Deep Learning model for a single basin, pretraining on a large and diverse dataset, followed by finetuning the pretrained model on your target basin is the way to go.

In [1]:
# Imports
from pathlib import Path
import pandas as pd
import torch
import sys
sys.path.append(r"C:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology")

from neuralhydrology.nh_run import start_run, eval_run, finetune

## Pretraining

In the first step, we need to pretrain our model on a large and possibly diverse dataset. Our target basin does not necessarily have to be a part of this dataset, but usually it should be better to include it.

For the sake of the demonstration, we will train an LSTM on the CAMELS US dataset and then finetune this model to a random basin. Note that it is possible to use other inputs during pretraining and finetuning, if additional embedding layers (before the LSTM) are used, which we will ignore for now. Furthermore, we will concentrate only on demonstrating the "how-to" rather than striving for best-possible performance. To save time and energy, we will only pretrain the model for a small number of epochs. When striving for the best possible performance, you should make sure that you pretrain the model as best as possible, before starting to finetune.

We will stick closely to the model and experimental setup from [Kratzert et al. (2019)](https://hess.copernicus.org/articles/23/5089/2019/hess-23-5089-2019.html). To summarize:
- A single LSTM layer with a hidden size of 128.
- Input sequences are 365 days and the prediction is made at the last timestep.
- We will use the same CAMELS attributes, as in the publication mentioned above, as additional inputs at every time step so that the model can learn different hydrological behaviors depending on the catchment properties.

For more details, take a look at the config print-out below.

**Note**
- The config file assumes that the CAMELS US dataset is stored under `data/CAMELS_US` (relative to the main directory of this repository) or a symbolic link exists at this location. Make sure that this folder contains the required subdirectories `basin_mean_forcing`, `usgs_streamflow` and `camels_attributes_v2.0`. If your data is stored at a different location and you can't or don't want to create a symbolic link, you will need to change the `data_dir` argument in the `531_basins.yml` config file that is located in the same directory as this notebook.
- By default, the config (`531_basins.yml`) assumes that you have a CUDA-capable NVIDIA GPU (see config argument `device`). In case you don't have any or you have one but want to train on the CPU, you can either change the config argument to `device: cpu` or pass `gpu=-1` to the `start_run()` function. Please note that training such a model on such a large dataset on CPU takes a very long time. 

In [5]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
    start_run(config_file=Path("hysets.yml"))

# fall back to CPU-only mode
else:
    start_run(config_file=Path("hysets.yml"), gpu=-1)

2023-01-24 12:16:06,965: Logging to c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_hysets_limited_2401_121606\output.log initialized.
2023-01-24 12:16:06,966: ### Folder structure created at c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_hysets_limited_2401_121606
2023-01-24 12:16:06,968: ### Run configurations for cudalstm_hysets_limited
2023-01-24 12:16:06,968: experiment_name: cudalstm_hysets_limited
2023-01-24 12:16:06,968: run_dir: c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_hysets_limited_2401_121606
2023-01-24 12:16:06,969: train_basin_file: hysets_basin_list.txt
2023-01-24 12:16:06,971: validation_basin_file: hysets_basin_list.txt
2023-01-24 12:16:06,971: test_basin_file: hysets_basin_list.txt
2023-01-24 12:16:06,972: train_start_date: 2006-10-01 00:00:00
2023-01-24 12:16:06,972: train_end_date: 2018-09-30 00:00:00
2023-0

C:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\neuralhydrology\datasetzoo\basedataset.py:460: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  per_basin_target_stds = torch.tensor([np.nanstd(obs, axis=1)], dtype=torch.float32)


100%|██████████| 2/2 [00:01<00:00,  1.45it/s]
2023-01-24 12:16:10,927: Setting learning rate to 0.0005
# Epoch 1: 100%|██████████| 33/33 [01:17<00:00,  2.34s/it, Loss: 0.1516]
2023-01-24 12:17:28,081: Epoch 1 average loss: 0.09095768472461989
# Validation: 100%|██████████| 2/2 [00:25<00:00, 12.86s/it]
2023-01-24 12:17:53,813: Stored results at c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_hysets_limited_2401_121606\validation\model_epoch001\validation_results.p
2023-01-24 12:17:53,814: Epoch 1 average validation loss: 0.06621 -- Median validation metrics: NSE: 0.29844
# Epoch 2: 100%|██████████| 33/33 [01:17<00:00,  2.35s/it, Loss: 0.0215]
2023-01-24 12:19:11,333: Epoch 2 average loss: 0.07265197079290044
# Validation: 100%|██████████| 2/2 [00:23<00:00, 11.53s/it]
2023-01-24 12:19:34,405: Stored results at c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_hysets_limited_2401_121606\val

We end with an okay'ish model that should be enough for the purpose of this demonstration. Remember we only train for a limited number of epochs here.

Next, we'll load the validation results into memory so we can select a basin to demonstrate how to finetune based on the model performance. 
Since the folder name is created dynamically (including the date and time of the start of the run) you will need to change the `run_dir` argument according to your local directory name. 

Here, we will select a random basin from the lower 50% of the NSE distribution, i.e. a basin where the NSE is below the median NSE. Usually, you'll see better performance gains for basins with lower model performance than for those where the base model is already really good.

In [2]:
# Load validation results from the last epoch
run_dir = Path("runs/cudalstm_hysets_limited_2401_121606/")
df = pd.read_csv(run_dir / "validation" / "model_epoch003" / "validation_metrics.csv", dtype={'basin': str})
df = df.set_index('basin')

# Compute the median NSE from all basins, where discharge observations are available for that period
print(f"Median NSE of the validation period {df['NSE'].median():.3f}")

# Select a random basins from the lower 50% of the NSE distribution
basin = df.loc[df["NSE"] < df["NSE"].median()].sample(n=1).index[0]

print(f"Selected basin: {basin} with an NSE of {df.loc[df.index == basin, 'NSE'].values[0]:.3f}")

Median NSE of the validation period 0.456
Selected basin: 9 with an NSE of 0.385


## Finetuning

Next, we will show how to perform finetuning for the basin selected above, based on the model we just trained. The function to use is `finetune` from `neuralhydrology.nh_run` if you want to train from within a script or notebook. If you want to start finetuning from the command line, you can also call the `nh-run` utility with the `finetune` argument, instead of e.g. `train` or `evaluate`.

The only thing required, similar to the model training itself, is a config file. This config however has slightly different requirements to a normal model config and works slightly different:
- The config has to contain the following two arguments:
    - `base_run_dir`: The path to the directory of the pre-trained model.
    - `finetune_modules`: Which parts of the pre-trained model you want to finetune. Check the documentation of each model class for a list of all possible parts. Often only parts, e.g. the output layer, are trained during finetuning and the rest is kept fixed. There is no general rule of thumb and most likely you will have to try both.
- Any additional argument contained in this config will overwrite the config argument of the pre-trained model. Everything _not_ specified will be taken from the pre-trained model. That is, you can e.g. specify a new basin file in the finetuning config (by `train_basin_file`) to finetune the pre-trained model on a different set of basins, or even just a single basin as we will do in this notebook. You can also change the learning rate, loss function, evaluation metrics and so on. The only thing you can not change are arguments that change the model architecture (e.g. `model`, `hidden_size` etc.), because this leads to errors when you try to load the pre-trained weights into the initialized model.

Let's have a look at the `finetune.yml` config that we prepared for this tutorial (you can find the file in the same directory as this notebook).

In [17]:
!cat finetune.yml

# --- Experiment configurations --------------------------------------------------------------------

# experiment name, used as folder name
experiment_name: cudalstm_531_basins_finetuned

# files to specify training, validation and test basins (relative to code root or absolute path)
train_basin_file: finetune_basin.txt
validation_basin_file: finetune_basin.txt
test_basin_file: finetune_basin.txt

# --- Training configuration -----------------------------------------------------------------------

# specify learning rates to use starting at specific epochs (0 is the initial learning rate)
learning_rate:
    0: 5e-4
    2: 5e-5	

# Number of training epochs
epochs: 10

finetune_modules:
- head
- lstm
base_run_dir: /home/frederik/Projects/neuralhydrology/examples/06-Finetuning/runs/cudalstm_531_basins_0901_135400

So out of the two arguments that are required, `base_run_dir` is still missing. We will add the argument from here and point at the directory of the model we just trained. Furthermore, we point to a new file for training, validation and testing, called `finetune_basin.txt`, which does not yet exist. We will create this file and add the basin we selected above as the only basin we want to use here. The rest are some changes to the learning rate and the number of training epochs as well as a new name. Also note that here, we train the full model, by selecting all model parts available for the `CudaLSTM` under `finetune_modules`.

In [18]:
# Add the path to the pre-trained model to the finetune config
with open("finetune.yml", "a") as fp:
    fp.write(f"base_run_dir: {run_dir.absolute()}")
    
# Create a basin file with the basin we selected above
with open("finetune_basin.txt", "w") as fp:
    fp.write(basin)

With that, we are ready to start the finetuning. As mentioned above, we have two options to start finetuning:
1. Call the `finetune()` function from a different Python script or a Jupyter Notebook with the path to the config.
2. Start the finetuning from the command line by calling

```bash
nh-run finetune --config-file /path/to/config.yml
```

Here, we will use the first option.

In [2]:
finetune(Path("hysets_finetune.yml"))

2023-01-24 12:46:57,129: Logging to c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_531_basins_finetuned_2401_124657\output.log initialized.
2023-01-24 12:46:57,130: ### Folder structure created at c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_531_basins_finetuned_2401_124657
2023-01-24 12:46:57,130: ### Start finetuning with pretrained model stored in c:\Users\everett\Documents\GitHub\neuralhydrology\neuralhydrology\examples\10-Hysets\runs\cudalstm_hysets_limited_2401_121606
2023-01-24 12:46:57,131: ### Run configurations for cudalstm_531_basins_finetuned
2023-01-24 12:46:57,131: batch_size: 256
2023-01-24 12:46:57,131: clip_gradient_norm: 1
2023-01-24 12:46:57,132: clip_targets_to_zero: ['q(mm/d)']
2023-01-24 12:46:57,132: commit_hash: 4774ca6
2023-01-24 12:46:57,133: data_dir: ..\..\data\hysets
2023-01-24 12:46:57,133: dataset: hysets
2023-01-24 12:46:57,134: device: cpu
2023-01-2

RuntimeError: The following attributes have a std of zero or NaN, which results in NaN's when normalizing the features. Remove the attributes from the attribute feature list and restart the run. 
Attributes: ['Elevation_m', 'Slope_deg', 'Land_Use_Forest_frac', 'Land_Use_Grass_frac', 'Land_Use_Wetland_frac', 'Land_Use_Water_frac', 'Land_Use_Urban_frac', 'Land_Use_Shrubs_frac', 'Land_Use_Crops_frac', 'precip_mean(mm/d)', 'precip_high(mm/d)', 'precip_mean(mm/mo)', 'precip_mean(mm/y)', 'precip_high_freq(d/yr)', 'precip_high_dur(d)', 'precip_low_freq(d/yr)', 'precip_low_dur(d)', 'baseflow_index(-)', 'q_mean(mm/d)', 'q_high(mm/d)', 'q_low(mm/d)', 'q_high_freq(d/yr)', 'q_high_dur(d)', 'q_low_dur(d)', 'q_95', 'q_5', 'runoff_ratio', 'q_adf(-)', 'q_adf_p(-)', 'q_adf_cv1(-)', 'q_adf_cv5(-)', 'q_adf_cv10(-)', 'q_seasonality(-)', 'tmean_seasonality(-)', 'tmean_alltime(C)', 'tmax_annual_mean(C)', 'tmin_annual_mean(C)']

Looking at the validation result, we can see an increase of roughly 0.05 NSE.

Last but not least, we will compare the pre-trained and the finetuned model on the test period. For this, we will make use of the `eval_run` function from `neuralhydrolgy.nh_run`. Alternatively, you could evaluate both runs from the command line by calling

```bash
nh-run evaluate --run-dir /path/to/run_directory/
```

In [20]:
eval_run(run_dir, period="test")

2022-01-09 14:16:09,586: Using the model weights from runs/cudalstm_531_basins_0901_135400/model_epoch003.pt
# Evaluation: 100%|██████████| 531/531 [02:09<00:00,  4.11it/s]
2022-01-09 14:18:18,959: Stored results at runs/cudalstm_531_basins_0901_135400/test/model_epoch003/test_results.p


Now we can call the `eval_run()` function as above, but pointing to the directory of the finetuned run. By default, this function evaluates the last checkpoint, which can be changed with the `epoch` argument. Here however, we use the default. Again, if you want to run this notebook locally, make sure to adapt the folder name of the finetune run.

In [23]:
finetune_dir = Path("runs/cudalstm_531_basins_finetuned_0901_141548")
eval_run(finetune_dir, period="test")

2022-01-09 14:19:06,488: Using the model weights from runs/cudalstm_531_basins_finetuned_0901_141548/model_epoch010.pt
# Evaluation: 100%|██████████| 1/1 [00:00<00:00,  4.27it/s]
2022-01-09 14:19:06,726: Stored results at runs/cudalstm_531_basins_finetuned_0901_141548/test/model_epoch010/test_results.p


Now let's look at the test period results of the pre-trained base model and the finetuned model for the basin that we chose above.

In [24]:
# load test results of the base run
df_pretrained = pd.read_csv(run_dir / "test/model_epoch003/test_metrics.csv", dtype={'basin': str})
df_pretrained = df_pretrained.set_index("basin")
    
# load test results of the finetuned model
df_finetuned = pd.read_csv(finetune_dir / "test/model_epoch010/test_metrics.csv", dtype={'basin': str})
df_finetuned = df_finetuned.set_index("basin")
    
# extract basin performance
base_model_nse = df_pretrained.loc[df_pretrained.index == basin, "NSE"].values[0]
finetune_nse = df_finetuned.loc[df_finetuned.index == basin, "NSE"].values[0]
print(f"Basin {basin} base model performance: {base_model_nse:.3f}")
print(f"Performance after finetuning: {finetune_nse:.3f}")

Basin 02112360 base model performance: 0.303
Performance after finetuning: 0.580


So we see roughly the same performance increase in the test period (slightly higher), which is great. However, note that a) our base model was not optimally trained (we stopped quite early) but also b) the finetuning settings were chosen rather randomly. From our experience so far, you can almost always get performance increases for individual basins with finetuning, but it is difficult to find settings that are universally applicable. However, this tutorial was just a showcase of how easy it actually is to finetune models with the NeuralHydrology library. Now it is up to you to experiment with it.